In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout,Dense,Input,Lambda
from tensorflow.keras.applications import MobileNetV2

In [14]:
batch_size=32
img_size=(128,128)
attack_dir = r"..\data\Detectedface\ImposterFace"
real_dir = r"..\data\Detectedface\ClientFace"
data_dir = r"..\data\Detectedface"

In [23]:
datagen=ImageDataGenerator(validation_split=0.2)

In [ ]:
# Training generator with augmentation (helps prevent overfitting)
train_datagen = ImageDataGenerator(
  
    rotation_range=5,         
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True,
    brightness_range=(0.7, 1.3),
    fill_mode='nearest'
)


# Validation generator WITHOUT augmentation
val_datagen = ImageDataGenerator(validation_split=0.2)

Found 10092 images belonging to 2 classes.
Found 2522 images belonging to 2 classes.


In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir,                   
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'           
)
validation_generator = val_datagen.flow_from_directory(
    data_dir,                   
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'        
)

In [ ]:
model=Sequential()
model.add(Input(shape=(128,128,3)))
model.add(Lambda(preprocess_input))
base=MobileNetV2(include_top=False,weights='imagenet',input_shape=(128,128,3))
base.trainable=False
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu",kernel_regularizer=l2(  0.01)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))



In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=3,          
    restore_best_weights=True  
)

checkpoint = ModelCheckpoint(
    'best_model.keras', 
    monitor='val_loss', 
    save_best_only=True
)


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hist=model.fit(train_generator,validation_data=validation_generator,epochs=15,verbose=2,callbacks=[early_stop,checkpoint])